In [35]:
from bs4 import BeautifulSoup
import requests
import time

In [18]:
url = 'https://burgerkingrus.ru/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}


In [19]:
page = requests.get(url, headers=headers)

In [89]:
# selenium

In [75]:
import time

# Инициализация драйвера браузера (в этом примере используется Chrome)
driver = webdriver.Chrome()

# Открытие веб-страницы
driver.get('https://burgerkingrus.ru/')

# Получение высоты страницы
total_height = driver.execute_script("return document.body.scrollHeight")

# Установка шага прокрутки и задержки
scroll_step = total_height / 20
scroll_delay = 0.5

# Медленная прокрутка страницы вниз
for i in range(100):
    driver.execute_script(f"window.scrollTo(0, {scroll_step * (i + 1)});")
    time.sleep(scroll_delay)

page = driver.page_source
# Закрытие браузера
driver.quit()

## bs4

In [249]:
# список исключений для категорий
exception_list = ['Новинки', 'Кинг Комбо',
                  'Кинг Сет', 'Кинг Топ Обед', 
                  'Купоны']

In [250]:
from bs4 import BeautifulSoup

# Предположим, что page_source - это ваш HTML
soup = BeautifulSoup(page, 'html.parser')

# Находим все блоки категорий
category_blocks = soup.find_all(class_='bk-category-block')

result_list = []

# Для каждого блока категории
for category_block in category_blocks:
    # Находим название категории
    category_title = category_block.find(class_='bk-category__title').text.strip().replace('\n', '')
    if category_title in exception_list: # скипаем бяку
        continue

    # Находим все блюда в этой категории
    dishes = category_block.find_all(class_='bk-dish-card')
    # Для каждого блюда
    for dish in dishes:
        # Находим название блюда
        dish_title = dish.find(class_='bk-dish-card__title').text.strip().replace('\n', '')

        # Находим цену блюда
        dish_price = dish.find(class_='bk-price').text.strip().replace('\n', '')

        # Выводим информацию
        result_list.append((dish_title,category_title, dish_price))
       # print(f'Category: {category_title}, Dish: {dish_title}, Price: {dish_price}')


In [251]:
len(result_list)

223

In [252]:
parsing_df = pd.DataFrame(result_list, columns=['item_name', 'item_category', 'item_price'])

# Вывод DataFrame
parsing_df

,item_name,item_category,item_price
0,Острый Сибирский Кинг,Острые бургеры,319.99
1,Острый Сибирский Кинг Двойной,Острые бургеры,429.99
2,Острый Сибирский Кинг с курицей,Острые бургеры,329.99
3,Острый Чикен Тар-Тар,Острые бургеры,219.99
4,Острый Цезарь Ролл Со Стрипсами,Острые бургеры,289.99
...,...,...,...
218,Салат Цезарь с креветками,Салаты,389.99
219,Жевательная резинка Орбит сладкая мята,Дополнительно,49.99
220,Бумажный пакет,Дополнительно,19.99
221,Жевательная резинка Орбит bubblemint,Дополнительно,49.99


In [253]:
parsing_df.to_csv('parsing_df.csv')

In [254]:
parsing_df.item_category.unique()

array(['Острые бургеры', 'Острые закуски', 'Острый картофель', 'Говядина',
       'Курица', 'Рыба', 'Креветки', 'Роллы', 'Соусы', 'Картофель',
       'Закуски', 'Холодные напитки', 'Пиво', 'Горячие напитки',
       'Молочные коктейли', 'Десерты', 'А4 Бокс', 'Салаты',
       'Дополнительно'], dtype=object)

In [256]:
# Группировка данных по названию товара и объединение категорий
grouped = parsing_df.groupby('item_name').agg({
    'item_category': lambda x: ', '.join(x.unique()),
    'item_price': 'first'
}).reset_index()


In [257]:
grouped.head()

,item_name,item_category,item_price
0,2 Соуса На Выбор,Соусы,79.99
1,2 сырных соуса,Соусы,79.99
2,3 Соуса На Выбор,Соусы,99.99
3,6 соусов на выбор,Соусы,199.99
4,Flash Up,Холодные напитки,89.99


## read data

In [258]:
train_df = pd.read_parquet('data/train_dataset_hackaton2023_train.gzip')

In [259]:
test_df = pd.read_parquet('data/hackaton2023_test.gzip')

In [260]:
test_dishes = set(test_df.dish_name.unique())

In [261]:
train_dishes = set(train_df.dish_name.unique())

In [262]:
unique_test_dishes = test_dishes - train_dishes

In [263]:
test_uniq_list = list(unique_test_dishes)

# comparing

In [462]:
from fuzzywuzzy import process
from tqdm import tqdm
import numpy as np

def match_name(name, list_names, min_score=0):
    # Находим наиболее похожее имя в list_names к данному name
    max_score = -1
    max_name = ""
    for name2 in list_names:
        score = process.extractOne(name, [name2])[1]
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
   # print(f'{name}-{max_name}-{max_score}')
    if max_score < min_score:
        return np.nan, np.nan
    return max_name, max_score

names = grouped['item_name'].tolist()

# Создаем новый DataFrame только с уникальными названиями продуктов
unique_train_df = pd.DataFrame(train_df['dish_name'].unique(), columns=['dish_name']) # point
updated_unique_train_df = pd.concat([unique_train_df, test_uniq_df], ignore_index=True)

# Применяем функцию к каждому уникальному имени продукта в train_df
# Добавляем progress bar с помощью tqdm
updated_unique_train_df['matched_name'] = [match_name(x, names, 87)[0] for x in updated_unique_train_df['dish_name']] # in tqdm(...)

# Теперь можно смержить два датафрейма по совпадающим именам
merged_df = pd.merge(updated_unique_train_df, grouped, left_on='matched_name', right_on='item_name', how='left')


In [463]:
len(merged_df)

1049

In [464]:
# Преобразование test_uniq_list в DataFrame
test_uniq_df = pd.DataFrame(test_uniq_list, columns=['dish_name'])

# Объединение unique_train_df с test_uniq_df
updated_unique_train_df = pd.concat([unique_train_df, test_uniq_df], ignore_index=True)

In [465]:
def categorize_dish(dish_name):
    dish_name = dish_name.lower()
    # Картофель
    if "карт." in dish_name:
        return "Картофель"
    
    # Десерты
    elif "сандэй" in dish_name or "маффин" in dish_name or 'мороженое' in dish_name or 'улитка' in dish_name:
        return "Десерты"

    # Соус
    elif 'соус' in dish_name:
        return 'Соусы'

    # Пиво
    elif "пиво" in dish_name:
        return "Пиво"

    # Молочные коктейли
    elif "шейк" in dish_name:
        return "молочные коктейли"

    # А4 Бокс
    elif "a4" in dish_name or 'а4' in dish_name:
        return "А4 Бокс"

    # Закуски
    elif "догпер" in dish_name or "чипсы" in dish_name or "начос" in dish_name or "сухарики" in dish_name:
        return "Закуски"

    # Холодные напитки
    elif "миринда" in dish_name or "пепси" in dish_name or "севен ап" in dish_name or "сок" in dish_name or \
         "адреналин" in dish_name or "эвервесс" in dish_name or "кола" in dish_name or \
         "липтон" in dish_name or "байкал" in dish_name or "мандарин" in dish_name or \
         "лимонад" in dish_name:
        return "Холодные напитки"

        # Курица [перед говядиной должна быть]
    elif "инди" in dish_name or "чикен" in dish_name or 'курицей' in dish_name or 'стрипс' in dish_name:
        return "Курица"

    # Говядина
    elif "сырбургер" in dish_name or "беконайзер" in dish_name or "стейкхаус" in dish_name or 'воппер' in dish_name or \
         "ангус" in dish_name or "конверт" in dish_name or "мясной" in dish_name or "тильзитер" in dish_name or 'итальяно':
        return "Говядина"

    # Дополнительно
    elif "антисептик" in dish_name or "пакет" in dish_name or "3d-патч" in dish_name or \
        "комикс" in dish_name or "игрушка" in dish_name or 'стикстер' in dish_name:
        return "Дополнительно"


    return np.nan  # Значение по умолчанию, если не совпадает ни с одной категорией


In [466]:
merged_df.loc[merged_df['item_price'].isna(), 'item_category'] = merged_df['dish_name'].apply(categorize_dish)

In [467]:
nan_merged_df = merged_df[merged_df['item_category'].isnull()]

In [468]:
nan_merged_df

,dish_name,matched_name,item_name,item_category,item_price


In [469]:
nan_merged_df.to_excel('nan.xlsx')

In [470]:
merged_df = merged_df.drop('matched_name', axis=1)

In [471]:
merged_df

,dish_name,item_name,item_category,item_price
0,Кинг Фри станд,Кинг Фри,Картофель,84.99
1,Чикен Тар-Тар,Чикен Тар-Тар,Курица,205.99
2,Соус Сырный,Соус Сырный,Соусы,49.99
3,Энергет.нап. Адреналин Раш,NaN,Холодные напитки,NaN
4,Латте (СТАНД.),Латте,Горячие напитки,69.99
...,...,...,...,...
1044,Up Латте Ваниль (БОЛ.) за 1руб,Латте,Горячие напитки,69.99
1045,Севен Ап XL 1л,NaN,Холодные напитки,NaN
1046,Up Чеддер Начос Кинг с курицей,NaN,Закуски,NaN
1047,"lim.Кола без сахара мал 0,4",Кола без сахара,Холодные напитки,109.99


In [472]:
train_df.shape

(12129384, 9)

In [473]:
# Предположим, что df1 - это ваш первый DataFrame, а df2 - второй
train_m_df = train_df.merge(merged_df, left_on='dish_name', right_on='dish_name', how='left')


In [474]:
train_m_df.item_category.isna().sum()

0

In [479]:
merged_df.to_csv('parsing_categories.csv')

In [476]:
merged_df.item_category.nunique()

29

In [477]:
merged_df.item_category.unique()

array(['Картофель', 'Курица', 'Соусы', 'Холодные напитки',
       'Горячие напитки', 'Говядина', 'Роллы', 'Закуски', 'Дополнительно',
       'Десерты',
       'Острые закуски, Острый картофель, Соусы, Картофель, Закуски',
       'Острые закуски, Закуски', 'А4 Бокс', 'молочные коктейли',
       'Креветки, Закуски', 'Острые бургеры, Говядина', 'Рыба',
       'Острые бургеры, Рыба',
       'Холодные напитки, Молочные коктейли, Десерты', 'Пиво',
       'Острые закуски, Острый картофель, Картофель, Закуски',
       'Острый картофель, Картофель', 'Острые бургеры, Роллы',
       'Креветки, Салаты', 'Холодные напитки, Пиво',
       'Острые закуски, Креветки, Закуски', 'Острые бургеры, Курица',
       'Картофель, А4 Бокс', 'Салаты'], dtype=object)

In [478]:
merged_df

,dish_name,item_name,item_category,item_price
0,Кинг Фри станд,Кинг Фри,Картофель,84.99
1,Чикен Тар-Тар,Чикен Тар-Тар,Курица,205.99
2,Соус Сырный,Соус Сырный,Соусы,49.99
3,Энергет.нап. Адреналин Раш,NaN,Холодные напитки,NaN
4,Латте (СТАНД.),Латте,Горячие напитки,69.99
...,...,...,...,...
1044,Up Латте Ваниль (БОЛ.) за 1руб,Латте,Горячие напитки,69.99
1045,Севен Ап XL 1л,NaN,Холодные напитки,NaN
1046,Up Чеддер Начос Кинг с курицей,NaN,Закуски,NaN
1047,"lim.Кола без сахара мал 0,4",Кола без сахара,Холодные напитки,109.99
